In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
#from keras.models  import Sequential
#from keras.layers  import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score


## Leitura dos dados

In [2]:
df = pd.read_excel('dados/Todos os resultados da Mega Sena  Rede Loteria.xlsx')
df.columns = df.iloc[0,:]
df = df.drop(0)

colunas = ['Conc', 'Data', '1a_dezena','2a_dezena','3a_dezena','4a_dezena', '5a_dezena','6a_dezena', 'Gan.', 'Premio','Apostas']
df.columns = colunas

df['1a_dezena'] = df['1a_dezena'].astype(int)
df['2a_dezena'] = df['2a_dezena'].astype(int)
df['3a_dezena'] = df['3a_dezena'].astype(int)
df['4a_dezena'] = df['4a_dezena'].astype(int)
df['5a_dezena'] = df['5a_dezena'].astype(int)
df['6a_dezena'] = df['6a_dezena'].astype(int)

df.head()

,Conc,Data,1a_dezena,2a_dezena,3a_dezena,4a_dezena,5a_dezena,6a_dezena,Gan.,Premio,Apostas
1,2669,16/12/2023,4,7,16,35,46,54,0,NaN,8163974
2,2668,14/12/2023,1,27,30,41,46,57,0,NaN,6123188
3,2667,12/12/2023,1,4,8,21,46,51,0,NaN,5403236
4,2666,09/12/2023,5,25,29,30,43,47,1,30781665.32,10479456
5,2665,07/12/2023,3,14,21,22,37,39,0,NaN,8471087


In [3]:
df.shape

(2669, 11)

## Verificar celulas vazias

In [4]:
df.isna().sum()

Conc            0
Data            0
1a_dezena       0
2a_dezena       0
3a_dezena       0
4a_dezena       0
5a_dezena       0
6a_dezena       0
Gan.            0
Premio       2065
Apostas      2202
dtype: int64

Algumas celulas vazias nas colunas de premio e apostas.

**Vamos fazer um modelo retirando essas colunas**

## Seleção de variáveis

In [5]:
df1 = df.drop(['Conc','Data','Premio','Apostas'], axis=1)
df1['Gan.'] = df1['Gan.'].astype(int) 
df1['Gan'] = np.where(df1['Gan.'] >= 1, 1, 0)
df1 = df1.drop('Gan.', axis=1)
df1.head()

,1a_dezena,2a_dezena,3a_dezena,4a_dezena,5a_dezena,6a_dezena,Gan
1,4,7,16,35,46,54,0
2,1,27,30,41,46,57,0
3,1,4,8,21,46,51,0
4,5,25,29,30,43,47,1
5,3,14,21,22,37,39,0


In [6]:
df1[df1['Gan'] >= 1]

,1a_dezena,2a_dezena,3a_dezena,4a_dezena,5a_dezena,6a_dezena,Gan
4,5,25,29,30,43,47,1
8,17,20,31,34,40,42,1
12,5,13,39,51,58,60,1
16,11,17,23,36,47,51,1
19,6,23,35,36,37,59,1
...,...,...,...,...,...,...,...
2646,1,8,14,28,33,43,1
2653,6,10,13,19,20,51,1
2659,15,25,37,38,58,59,1
2667,10,11,29,30,36,47,1


In [7]:
df1.dtypes

1a_dezena    int32
2a_dezena    int32
3a_dezena    int32
4a_dezena    int32
5a_dezena    int32
6a_dezena    int32
Gan          int32
dtype: object

In [8]:
df1.head()

,1a_dezena,2a_dezena,3a_dezena,4a_dezena,5a_dezena,6a_dezena,Gan
1,4,7,16,35,46,54,0
2,1,27,30,41,46,57,0
3,1,4,8,21,46,51,0
4,5,25,29,30,43,47,1
5,3,14,21,22,37,39,0


In [9]:
df1.groupby(['1a_dezena','2a_dezena', '3a_dezena', '4a_dezena', '5a_dezena', '6a_dezena']).size().sort_values(ascending=False)

# Até agora nunca tivemos um jogo repetido com as dezenas na mesma ordem

1a_dezena  2a_dezena  3a_dezena  4a_dezena  5a_dezena  6a_dezena
1          2          3          11         28         43           1
10         14         17         18         23         30           1
                                 25         32         39           1
                      22         41         42         53           1
                      25         29         33         57           1
                                                                   ..
4          20         24         35         47         59           1
                      30         41         47         58           1
           21         25         33         36         46           1
                      27         29         42         47           1
50         51         56         57         58         59           1
Length: 2669, dtype: int64

## Construção do modelo

In [10]:
X = df1.drop('Gan', axis=1)
y = df1['Gan']

X_train,X_test, y_train, y_test = train_test_split(X, y, test_size = .33)

In [26]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=25, activation='relu'))
ann.add(tf.keras.layers.Dense(units=25, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
ann.fit(X_train, y_train, batch_size=20, epochs=100)

Epoch 1/100
90/90 [==============================] - 1s 987us/step - loss: 0.8964 - accuracy: 0.7036
Epoch 2/100
90/90 [==============================] - 0s 970us/step - loss: 0.5869 - accuracy: 0.7388
Epoch 3/100
90/90 [==============================] - 0s 967us/step - loss: 0.5598 - accuracy: 0.7662
Epoch 4/100
90/90 [==============================] - 0s 972us/step - loss: 0.5383 - accuracy: 0.7718
Epoch 5/100
90/90 [==============================] - 0s 976us/step - loss: 0.5361 - accuracy: 0.7696
Epoch 6/100
90/90 [==============================] - 0s 960us/step - loss: 0.5380 - accuracy: 0.7696
Epoch 7/100
90/90 [==============================] - 0s 978us/step - loss: 0.5402 - accuracy: 0.7746
Epoch 8/100
90/90 [==============================] - 0s 981us/step - loss: 0.5458 - accuracy: 0.7724
Epoch 9/100
90/90 [==============================] - 0s 974us/step - loss: 0.5284 - accuracy: 0.7735
Epoch 10/100
90/90 [==============================] - 0s 961us/step - loss: 0.5360 - accura

90/90 [==============================] - 0s 903us/step - loss: 0.5176 - accuracy: 0.7735
Epoch 82/100
90/90 [==============================] - 0s 885us/step - loss: 0.5185 - accuracy: 0.7729
Epoch 83/100
90/90 [==============================] - 0s 920us/step - loss: 0.5190 - accuracy: 0.7729
Epoch 84/100
90/90 [==============================] - 0s 932us/step - loss: 0.5184 - accuracy: 0.7735
Epoch 85/100
90/90 [==============================] - 0s 906us/step - loss: 0.5172 - accuracy: 0.7729
Epoch 86/100
90/90 [==============================] - 0s 880us/step - loss: 0.5190 - accuracy: 0.7729
Epoch 87/100
90/90 [==============================] - 0s 922us/step - loss: 0.5167 - accuracy: 0.7746
Epoch 88/100
90/90 [==============================] - 0s 900us/step - loss: 0.5214 - accuracy: 0.7740
Epoch 89/100
90/90 [==============================] - 0s 892us/step - loss: 0.5162 - accuracy: 0.7729
Epoch 90/100
90/90 [==============================] - 0s 919us/step - loss: 0.5154 - accuracy: 

In [12]:
#%%time
#
#import tensorflow as tf
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score
#
## Supondo que você tenha seus dados de treino e teste definidos como X_train, y_train, X_test, y_test
#
## Defina os diferentes conjuntos de parâmetros que deseja testar
#import numpy as np
#
## Defina o número total de conjuntos de parâmetros desejados
#num_parametros = 50
#
## Defina os intervalos para cada parâmetro
#intervalo_units1 = np.linspace(8, 32, num_parametros)
#intervalo_units2 = np.linspace(8, 32, num_parametros)
##intervalo_epochs = np.linspace(150, 300, num_parametros)
#
## Lista para armazenar os conjuntos de parâmetros
#parametros = []
#
## Crie os conjuntos de parâmetros com base nos intervalos
#for i in range(num_parametros):
#    params = {
#        'units1': int(intervalo_units1[i]),
#        'units2': int(intervalo_units2[i]),
#        'epochs': 100,
#    }
#    parametros.append(params)
#
#
## Lista para armazenar os modelos e suas métricas
#modelos_e_metricas = []
#
#for i, params in enumerate(parametros):
#    # Crie um modelo com base nos parâmetros atuais
#    ann = tf.keras.models.Sequential()
#    ann.add(tf.keras.layers.Dense(units=params['units1'], activation='relu'))
#    ann.add(tf.keras.layers.Dense(units=params['units2'], activation='relu'))
#    ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
#    ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#    
#    # Treine o modelo
#    ann.fit(X_train, y_train, batch_size=20, epochs=params['epochs'])
#    
#    # Faça previsões no conjunto de teste
#    y_pred = ann.predict(X_test)
#    y_pred = (y_pred > 0.3)
#    
#    # Avalie o desempenho do modelo e armazene as métricas
#    acuracia = accuracy_score(y_test, y_pred)
#    
#    # Adicione o modelo e métricas à lista com um nome único
#    modelos_e_metricas.append({'modelo': ann, 'acuracia': acuracia, 'parametros': params, 'nome': f'ann{i + 1}'})
#
## Agora, você pode acessar os modelos pelo nome (por exemplo, modelos_e_metricas[0]['modelo'], modelos_e_metricas[1]['modelo'], etc.)
#

**melhores parametros 25,25**

In [13]:
#t = pd.DataFrame(modelos_e_metricas)
#t.sort_values('acuracia', ascending = False)

## Resultados 

In [14]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.25)

#pred_array = 1 * y_pred.reshape(len(y_pred), 1)
#test_array = y_test.values.reshape(len(y_test), 1)

#np.concatenate([pred_array, test_array], axis=1)



28/28 [==============================] - 0s 736us/step


In [15]:
cm = confusion_matrix(y_test, y_pred) 
#vn, fp(rede disse que é 1, mas é 0), fn (rede disse que é 0, mas é 1), vp

cm

array([[534, 148],
       [134,  65]], dtype=int64)

In [16]:
accuracy_score(y_test, y_pred)

0.6799091940976163

In [17]:
y_pred


array([[False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [


# Oversampling e Undersampling


In [18]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Exemplo de Oversampling
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# Exemplo de Undersampling
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

#modelo
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=25, activation='relu'))
ann.add(tf.keras.layers.Dropout(0.5))  # Exemplo de camada de dropout
ann.add(tf.keras.layers.Dense(units=25, activation='relu'))
ann.add(tf.keras.layers.Dropout(0.5))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#monitoramento
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

#treinamento
ann.fit(X_resampled, y_resampled, batch_size=20, epochs=100)


Epoch 1/100
41/41 [==============================] - 1s 1ms/step - loss: 14.5164 - accuracy: 0.5074 - recall: 0.5309 - precision: 0.5071
Epoch 2/100
41/41 [==============================] - 0s 1ms/step - loss: 9.3468 - accuracy: 0.4765 - recall: 0.4691 - precision: 0.4762
Epoch 3/100
41/41 [==============================] - 0s 1ms/step - loss: 6.9985 - accuracy: 0.4605 - recall: 0.4938 - precision: 0.4630
Epoch 4/100
41/41 [==============================] - 0s 1ms/step - loss: 4.8085 - accuracy: 0.5074 - recall: 0.5580 - precision: 0.5067
Epoch 5/100
41/41 [==============================] - 0s 1ms/step - loss: 3.3321 - accuracy: 0.4914 - recall: 0.5037 - precision: 0.4916
Epoch 6/100
41/41 [==============================] - 0s 1ms/step - loss: 2.2519 - accuracy: 0.5086 - recall: 0.5506 - precision: 0.5080
Epoch 7/100
41/41 [==============================] - 0s 1ms/step - loss: 2.0868 - accuracy: 0.4691 - recall: 0.5235 - precision: 0.4722
Epoch 8/100
41/41 [============================

41/41 [==============================] - 0s 1ms/step - loss: 0.6957 - accuracy: 0.4864 - recall: 0.5506 - precision: 0.4880
Epoch 61/100
41/41 [==============================] - 0s 1ms/step - loss: 0.6915 - accuracy: 0.5198 - recall: 0.6099 - precision: 0.5167
Epoch 62/100
41/41 [==============================] - 0s 1ms/step - loss: 0.6983 - accuracy: 0.5025 - recall: 0.6148 - precision: 0.5020
Epoch 63/100
41/41 [==============================] - 0s 1ms/step - loss: 0.6946 - accuracy: 0.4753 - recall: 0.6000 - precision: 0.4802
Epoch 64/100
41/41 [==============================] - 0s 1ms/step - loss: 0.6966 - accuracy: 0.5099 - recall: 0.7062 - precision: 0.5071
Epoch 65/100
41/41 [==============================] - 0s 1ms/step - loss: 0.6941 - accuracy: 0.5074 - recall: 0.5037 - precision: 0.5075
Epoch 66/100
41/41 [==============================] - 0s 1ms/step - loss: 0.6911 - accuracy: 0.5086 - recall: 0.4642 - precision: 0.5095
Epoch 67/100
41/41 [==============================] - 

In [19]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.6)

cm = confusion_matrix(y_test, y_pred) 
#vn, fp(rede disse que é 1, mas é 0), fn (rede disse que é 0, mas é 1), vp

cm

28/28 [==============================] - 0s 684us/step


array([[682,   0],
       [199,   0]], dtype=int64)

In [20]:
#numero_sorteio = [[3,6,11,27,28,46]]

numero_sorteio = [[2, 7, 13, 47, 52, 56]]

y_predict = pd.DataFrame(numero_sorteio)
y_predict

,0,1,2,3,4,5
0,2,7,13,47,52,56


In [21]:
# Fazendo a predição - Classes
predict_class = ann.predict(y_predict)
print("1 = Tem chance de ganhar / 0 = Não tem chance de ganhar")
print("\n")
print("Previsão Modelo: ",predict_class[0][0])

1/1 [==============================] - 0s 44ms/step
1 = Tem chance de ganhar / 0 = Não tem chance de ganhar


Previsão Modelo:  0.51149446


In [22]:
numero_sorteio = [[17, 32, 58, 43, 54, 5],
                 [10, 28, 46, 3,51,19],
                 [27,50,21,35,47,1],
                 [36,15,5,48,22,39],
                 [4,5,59,47,13,52],
                 [4,5,59,47,13,53],
                 [4,5,59,47,13,54],
                 [4,5,59,47,13,55]]
numero_sorteio_ordenado = [sorted(sorteio) for sorteio in numero_sorteio]

y_predict = pd.DataFrame(numero_sorteio_ordenado)

# Achando a probabilidade - Proba
predict_proba = ann.predict(y_predict)
print("Qual a probabilidade do número sair?")
print("\n")
for index, row in y_predict.iterrows():
    print("Probabilidade de",row.tolist(),"ganhar:", round((predict_proba[index][0]*100),2), "%")

1/1 [==============================] - 0s 16ms/step
Qual a probabilidade do número sair?


Probabilidade de [5, 17, 32, 43, 54, 58] ganhar: 51.15 %
Probabilidade de [3, 10, 19, 28, 46, 51] ganhar: 51.15 %
Probabilidade de [1, 21, 27, 35, 47, 50] ganhar: 51.15 %
Probabilidade de [5, 15, 22, 36, 39, 48] ganhar: 51.15 %
Probabilidade de [4, 5, 13, 47, 52, 59] ganhar: 51.15 %
Probabilidade de [4, 5, 13, 47, 53, 59] ganhar: 51.15 %
Probabilidade de [4, 5, 13, 47, 54, 59] ganhar: 51.15 %
Probabilidade de [4, 5, 13, 47, 55, 59] ganhar: 51.15 %


In [29]:
numero_sorteio = [[4,7,10,21,33,45]]
numero_sorteio_ordenado = [sorted(sorteio) for sorteio in numero_sorteio]

y_predict = pd.DataFrame(numero_sorteio_ordenado)

# Achando a probabilidade - Proba
predict_proba = ann.predict(y_predict)
print("Qual a probabilidade do número sair?")
print("\n")
for index, row in y_predict.iterrows():
    print("Probabilidade de",row.tolist(),"ganhar:", round((predict_proba[index][0]*100),2), "%")

1/1 [==============================] - 0s 16ms/step
Qual a probabilidade do número sair?


Probabilidade de [4, 7, 10, 21, 33, 45] ganhar: 22.36 %


In [27]:
import random


probabilidade_boa   = 99
probabilidade_atual = 0

# Gerando list com as dezenas sorteadas
dezenas_sorteadas = df1[['1a_dezena','2a_dezena', '3a_dezena', '4a_dezena', '5a_dezena', '6a_dezena']].values.tolist()

# Gera sequencia de numeros até que a probabilidade seja maior ou igual que 99%
while(probabilidade_atual < probabilidade_boa):
    
    # Gera sequencia de número 
    dezenas_megasena = random.sample(range(1, 60), 6)
    
    # Numero gerado já foi sorteado?
    if not dezenas_megasena in dezenas_sorteadas:

        # Valida qual a probabilidade da seguência ser sorteada
        probabilidade_atual = int(ann.predict(pd.DataFrame([dezenas_megasena]))[0][0]*100)
    

print("Probabilidade de {0} % -> Dezenas: {1}".format(probabilidade_atual, sorted(dezenas_megasena))) 

1/1 [==============================] - 0s 15ms/step
Probabilidade de 99 % -> Dezenas: [1, 2, 19, 25, 29, 53]
